In [1]:
from pdbecif.mmcif_io import CifFileReader
import pandas as pd
import numpy as np
import numpy.linalg as LA
from pdb_toolkit import *

Read domain and rotation axis info

In [2]:
df_sum_rotation   = pd.read_csv("main_chain_rotation_sum.csv")
df_domain_s1a     = pd.read_csv("../Domain_definition/serca1a_each_domain.csv")
df_domain_s2b     = pd.read_csv("../Domain_definition/serca2b_each_domain.csv")
df_domain_spca1Ca = pd.read_csv("../Domain_definition/spca1_each_domain.csv")

add entry name to df_domains

In [3]:
df_domain_s1a["entry"]     = "s1a"
df_domain_s2b["entry"]     = "s2b"
df_domain_spca1Ca["entry"] = "spca1Ca"

Concatenate domain into a single df

In [4]:

df_domain_concat = pd.concat([df_domain_s1a, df_domain_s2b, df_domain_spca1Ca])

In [5]:
E1_ATP_list = ["6LLE", "3AR2", "Mb_Ca_2"]
E2P_list    = ["6LLY", "3B9B", "E2P_Ca" ]
entry_list  = ["s2b" , "s1a" , "spca1Ca"]
df_list     = []

for i in range(len(entry_list)):
    cif_E1ATP = CifFileReader().read(f"./{E1_ATP_list[i]}.cif")
    df_E1ATP = extract_coor(cif_E1ATP,f"{E1_ATP_list[i]}")
    df_E1ATP_valid = valid_resi(df_E1ATP)
    df_E1ATP_alphaC = alpha_C(df_E1ATP_valid)
    
    cif_E2P = CifFileReader().read(f"./{E2P_list[i]}.cif")
    df_E2P = extract_coor(cif_E2P,f"{E2P_list[i]}")
    df_E2P_valid = valid_resi(df_E2P)
    df_E2P_alphaC = alpha_C(df_E2P_valid)
    
    # Rename columns
    df_E1ATP_renamed = df_E1ATP_alphaC.rename(columns={"alpha_x"  : "E1ATP_alpha_x",
                                                       "alpha_y"  : "E1ATP_alpha_y",
                                                       "alpha_z"  : "E1ATP_alpha_z"},inplace=True)
    
    df_E2P_renamed   = df_E2P_alphaC.rename(columns={"alpha_x"  : "E2P_alpha_x",
                                                     "alpha_y"  : "E2P_alpha_y",
                                                     "alpha_z"  : "E2P_alpha_z"},inplace=True)
    # Merge E2P to E1ATP
    df_merged = pd.merge(df_E1ATP_alphaC, df_E2P_alphaC,
                        how="left",
                        on=["residue_chain", "residue_ID", "residue_seq","atom_ID"])
    # Add an entry column
    df_merged["entry"] = f"{entry_list[i]}"

    # Select only chain A residues
    df_chainA = df_merged.loc[df_merged["residue_chain"] == "A"]
    
    # Append df_merge to df_list
    df_list.append(df_chainA)

concetenate dfs in df_list to a single daraframe

In [6]:

df_concat = pd.concat(df_list)

Add domain info to each residues

In [7]:

df_resi_withDomain = pd.merge(df_concat, df_domain_concat,
                              how="left",
                              on=["entry", "residue_seq"])

df_resi_withDomain_clean = df_resi_withDomain.dropna().reset_index(drop=True)


In [8]:
# Add rotation axis coordinates to each residue based on their domain info.
df_withAxisCoor = pd.merge(df_resi_withDomain_clean, 
                           df_sum_rotation[["entry","domain","center_of_axis_x","center_of_axis_y","center_of_axis_z"]],
                           how="left",
                           on=["entry","domain"])

# Remove TM6-7 loop
df_withAxisCoor_clean = df_withAxisCoor.dropna().reset_index(drop=True)

## Calculate the angle

In [9]:
point_center = np.array([df_withAxisCoor_clean["center_of_axis_x"],df_withAxisCoor_clean["center_of_axis_y"],df_withAxisCoor_clean["center_of_axis_z"]]).transpose()
point_E1ATP  = np.array([df_withAxisCoor_clean["E1ATP_alpha_x"],   df_withAxisCoor_clean["E1ATP_alpha_y"],   df_withAxisCoor_clean["E1ATP_alpha_z"]]).transpose()
point_E2P    = np.array([df_withAxisCoor_clean["E2P_alpha_x"],     df_withAxisCoor_clean["E2P_alpha_y"],     df_withAxisCoor_clean["E2P_alpha_z"]]).transpose()

In [10]:
angle_array = np.array([])

for i in range(0,len(point_E2P)):
    c_to_E1ATP = point_E1ATP[i] - point_center[i]
    c_to_E2P   = point_E2P[i]   - point_center[i]
    cosine_angle = np.dot(c_to_E1ATP, c_to_E2P) / (np.linalg.norm(c_to_E1ATP) * np.linalg.norm(c_to_E2P))
    angle = np.arccos(cosine_angle)
    angle_array = np.append(angle_array,np.degrees(angle))

In [11]:
df_withAxisCoor_clean["angle"] = pd.Series(angle_array)

## Take a quick look of the data

In [12]:
df_withAxisCoor_clean.groupby("domain").describe().T

domain               A_domain    N_domain    P_domain       TM1-2       TM3-4  \
residue_seq count  498.000000  670.000000  496.000000  203.000000  238.000000   
            mean   144.927711  471.982090  597.905242   89.532020  289.361345   
            std     77.380523   67.631373  125.041816   21.996910   23.185086   
            min      1.000000  358.000000  329.000000   48.000000  247.000000   
            25%     53.250000  414.000000  584.750000   72.000000  270.000000   
...                       ...         ...         ...         ...         ...   
angle       min      4.275390    5.774733   11.423767   14.535307    4.381856   
            25%     65.835458   51.755326   19.439337   24.272965   14.222343   
            50%     82.522319   61.092287   20.462787   28.977283   16.904650   
            75%     92.122912   68.105235   22.342853   34.396869   20.670547   
            max    129.770101  155.040068   25.944604   45.029592   48.183638   

domain                  TM5-6      TM7-10  
residue_seq count  212.000000  459.000000  
            mean   755.349057  889.980392  
            std     34.143562   55.520971  
            min    682.000000  773.000000  
            25%    734.750000  848.500000  
...                       ...         ...  
angle       min      0.140804    0.006847  
            25%      2.552455    0.212345  
            50%      4.612481    1.020407  
            75%      7.988496    2.031770  
            max     24.133903   17.818611  

[88 rows x 7 columns]

## Save the result

In [13]:
df_withAxisCoor_clean.to_csv("output/main_chain_rotation_angle.csv",index=False)